In [6]:
###### 최근 영업일 기준 데이터 받기 #####
import requests as rq
from bs4 import BeautifulSoup
import re

url = 'https://finance.naver.com/sise/sise_deposit.naver'
data = rq.get(url)
data_html = BeautifulSoup(data.content, "html.parser")

#똑같은 부분이 여러개 위치할 때는 select_one() 사용할 것 -> 맨 위 데이터만 선택
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah'
).text


#숫자만 뽑아내기
biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)



###### 티커 크롤링 #####
from io import BytesIO
import pandas as pd

##코스피##
#post 방식이므로 url과 쿼리가 필요
#url은 generate.cmd의 request url, 쿼리는 Payload탭의 Form data 부분
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day, #위에서 구한 최근 영업일
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

#리퍼러를 남겨서 웹사이트 방문 흔적을 남겨야 데이터를 받을 수 있음
#generate.cmd에서 referer 부분을 복붙하면 됨
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

#otp 발급
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

#download.cmd의 request URL부분
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'

#파일 다운
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)

#csv의 html버전
#print(down_sector_stk.content)

#BytestIO를 이용해 바이너리스트림 형태로 변경
#한국 거래소 데이터는 EUC-KR 형태로 인코딩 되어있음
#read_csv 함수는 디폴트로 UTF-8 인코딩이라서 EUC-KR로 바꿔줘야함
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding = 'EUC-KR')

##코스닥##
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_ksq = {
    'mktId': 'KSQ',
    'trdDd': biz_day, #위에서 구한 최근 영업일
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

#otp 발급
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

#download.cmd의 request URL부분
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'

#파일 다운
down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)

#csv의 html버전
#print(down_sector_strk.content)

sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding = 'EUC-KR')

#print(sector_ksq)


#코스피 코스닥 합치기
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)

#종목명에 있는 공백 제거
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
#기준일 추가
krx_sector['기준일'] = biz_day

#print(krx_sector)



####개별 종목 지표 크롤링####
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}

headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

#otp 발급
otp_data = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

#download.cmd의 request URL부분
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'

#파일 다운
krx_ind = rq.post(down_url, {'code': otp_data}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding = 'EUC-KR')

#종목명에 있는 공백 제거
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
#기준일 추가
krx_ind['기준일'] = biz_day


## 다운 받은 데이터들 정리 ##


#종목 지표와 섹터 데이터 합치기
#기준이 되는 열로 합쳐야 함 -> on 뒤에 있는 것들이 기준
kor_ticker = pd.merge(krx_sector,
                      krx_ind, 
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(), how= 'outer'
                      )




## 일반적인 종목과 스팩, 우선주, 리츠, 기타 주식을 구분 ##
#스팩 종목
kor_ticker[kor_ticker['종목명'].str.contains('스펙|제[0-9]+호')]['종목명']

#우선주
#국내 종목 중 종목코드 끝이 0이면 보통주, 0이 아니면 우선주
kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명']

#리츠
#'리츠'로 끝나는 종목명
kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]

import numpy as np

#두 데이터에 공통으로 존재하지 않는 종목 찾기
#symmetric_difference()는 하나의 집함에만 존재하는 부분을 찾음
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스펙|제[0-9]+호'), '스펙',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff), '기타',
                                                         '보통주'))))

print(kor_ticker)

        종목코드      종목명    시장구분     업종명      종가    대비   등락률           시가총액  \
0     095570   AJ네트웍스   KOSPI    서비스업    4310    40  0.94   201804091450   
1     006840    AK홀딩스   KOSPI    기타금융   19000  -150 -0.78   251703659000   
2     027410      BGF   KOSPI    기타금융    3635   -20 -0.55   347930535285   
3     282330   BGF리테일   KOSPI     유통업  154500 -1800 -1.15  2670363477000   
4     138930  BNK금융지주   KOSPI    기타금융    6850    80  1.18  2206305800300   
...      ...      ...     ...     ...     ...   ...   ...            ...   
2617  024060     흥구석유  KOSDAQ      유통    5160   130  2.58    77400000000   
2618  010240       흥국  KOSDAQ   기계·장비    6400   120  1.91    78865254400   
2619  189980   흥국에프엔비  KOSDAQ  음식료·담배    2425    15  0.62    97334230475   
2620  037440       희림  KOSDAQ   기타서비스    8890   210  2.42   123770802750   
2621  238490       힘스  KOSDAQ     반도체    6750  -200 -2.88    76357593000   

           기준일      EPS    PER   선행 EPS  선행 PER      BPS   PBR   주당배당금  배당수익률  \
0     

In [7]:
#kor_ticker.to_csv('C:\\Users\\dlstj\\Desktop\\stock_ai_project\\practical\\kor_ticker.csv', encoding='EUC-KR')

In [8]:
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ','')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가', '시가총액',
                         '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})

In [10]:
print(kor_ticker)

        종목코드      종목명    시장구분      종가           시가총액       기준일      EPS  \
0     095570   AJ네트웍스   KOSPI    4310   201804091450  20230904    201.0   
1     006840    AK홀딩스   KOSPI   19000   251703659000  20230904     None   
2     027410      BGF   KOSPI    3635   347930535285  20230904    247.0   
3     282330   BGF리테일   KOSPI  154500  2670363477000  20230904  11203.0   
4     138930  BNK금융지주   KOSPI    6850  2206305800300  20230904   2404.0   
...      ...      ...     ...     ...            ...       ...      ...   
2617  024060     흥구석유  KOSDAQ    5160    77400000000  20230904    183.0   
2618  010240       흥국  KOSDAQ    6400    78865254400  20230904    740.0   
2619  189980   흥국에프엔비  KOSDAQ    2425    97334230475  20230904    309.0   
2620  037440       희림  KOSDAQ    8890   123770802750  20230904    567.0   
2621  238490       힘스  KOSDAQ    6750    76357593000  20230904     None   

        선행EPS      BPS   주당배당금 종목구분  
0       472.0   8076.0   270.0  보통주  
1        None  41948.0 

In [13]:
#mysql로 데이터 옮기기
import pymysql

con = pymysql.connect(user='root',
                       passwd = '8019',
                       host = '127.0.0.1',
                       db = 'stock_db',
                       charset = 'utf8')

mycursor = con.cursor()
query = f"""
    insert into kor_ticker (종목코드, 종목명, 시장구분, 종가, 시가총액, 기준일, EPS, 선행EPS, BPS, 주당배당금,종목구분)
    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) as new
    on duplicate key update
    종목명=new.종목명, 시장구분=new.시장구분, 종가=new.종가, 시가총액=new.시가총액, EPS=new.EPS, 선행EPS=new.선행EPS,
    BPS=new.BPS, 주당배당금=new.주당배당금, 종목구분=new.종목구분;
"""
#upsert 쿼리 작성

args = kor_ticker.values.tolist()
mycursor.executemany(query, args)
con.commit()

con.close()